# Markow Text generator

### Solution by: Akshay Punwatkar(AP509) and Ishan Gupta (IG55)

In [34]:
import numpy as np
import nltk
from collections import defaultdict 
from pprint import pprint
import operator
import time
import hashlib

In [2]:
#!pip install nltk
#nltk.download('gutenberg')

In [69]:
class autoComplete():
    def __init__(self):
        self.corpus = []
        self.n = 0
        self.corpus_hash = 0
        
    def clean_corpus(self):
        self.corpus[:] = [item.strip('"') for item in self.corpus if not 
                     (item in ['[',']',"*","-",";",":",".--","--",",",";--"] or item.isdigit())]
        
    def create_n_gram_array(self, n):
        n_gram_size = n
        n_gram_array = []
        for i in range(len(self.corpus)-n_gram_size+1):
            n_gram_array.append(corpus[i:(i+n_gram_size)])
        n_gram_array = np.array(n_gram_array)
        self.n_gram_array_list.append(n_gram_array)
        return self.n_gram_array_list
    
    def create_n_gram_dict(self, n):
        for n_gram in self.n_gram_array_list[n-2]:
            self.n_gram_dict[n][tuple(n_gram[:-1])][n_gram[-1]] += 1
    
    def create_n_gram(self,corpus,n):
        self.corpus_hash = hashlib.sha224(" ".join(corpus).encode('utf-8')).hexdigest()
        self.n = n
        self.corpus = corpus
        self.n_gram_array_list = []
        self.n_gram_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(int))) 
        self.clean_corpus()
        for i in range(2,n+1):
            self.create_n_gram_array(i)
            self.create_n_gram_dict(i)
            
    def finish_sentence(self, sentence, n, corpus, deterministic=False):
        """
        Input:-
            sentence     : A sentence [list of tokens] that we’re trying to build on
            n [int]      : The length of n-grams to use for prediction, and
            corpus [list]: Source corpus [list of tokens]
            deterministic: Flag indicating whether the process should be deterministic [bool]
    
        If deterministic is true ; Choose at each step the single most probable next token. 
                               When two tokens are equally probable, choose the lesser one (according to Python).
        If deterministic is false; Draw the next word randomly from the appropriate distribution. Use stupid backoff and no smoothing.
    
        Output:-
            Returns an extended sentence until the first ., ?, or ! is found OR until it has 15 total tokens
        """
        
        if self.n < n or self.corpus_hash != hashlib.sha224(" ".join(corpus).encode('utf-8')).hexdigest():
            self.create_n_gram(corpus, n)
            
        self.w_in_sentc = [w.lower() for w in sentence]
        for i in range(15):
            suggest_word = self.w_in_sentc[-1]
            for i in range(n,1,-1):
                n_gram_key = tuple(self.w_in_sentc[(-i+1):])

                if n_gram_key in self.n_gram_dict[i].keys():
                    if deterministic == True:
                        most_probab = max(self.n_gram_dict[i][n_gram_key].values())
                        opts = []
                        for k,v in self.n_gram_dict[i][n_gram_key].items():
                            if v == most_probab:
                                opts.append(k) 
                        suggest_word = np.sort(opts)[0]
                        break
                    else:
                        suggest_word = np.random.choice(list(self.n_gram_dict[i][n_gram_key].keys()))
                        break
            self.w_in_sentc.append(suggest_word)  
            if suggest_word in [".","?","!"]:
                return self.w_in_sentc
            
        return self.w_in_sentc

In [70]:
#test case
sentence = ['she', 'was', 'not'] 
n = 3
corpus = [w.lower() for w in nltk.corpus.gutenberg.words('austen-sense.txt')]
deterministic = True
ac = autoComplete()
#expected output:
#[’she’, ’was’, ’not’, ’in’, ’the’, ’world’, ’.’]

In [71]:
ac.finish_sentence(sentence, n, corpus, deterministic=True)

['she', 'was', 'not', 'in', 'the', 'world', '.']

In [74]:
ac.finish_sentence(["akshay","was"], 3, corpus, deterministic=True)

['akshay', 'was', 'not', 'in', 'the', 'world', '.']

In [75]:
ac.finish_sentence(["akshay","was"], 2, corpus, deterministic=True)

['akshay',
 'was',
 'not',
 'be',
 'a',
 'very',
 'well',
 'as',
 'she',
 'was',
 'not',
 'be',
 'a',
 'very',
 'well',
 'as',
 'she']

In [76]:
ac.finish_sentence(["akshay","was"], 3, corpus, deterministic=False)

['akshay',
 'was',
 'very',
 'unwilling',
 'to',
 'run',
 'the',
 'risk',
 'of',
 'being',
 'too',
 'weak',
 'for',
 'conversation',
 'submitted',
 'readily',
 'to']

In [77]:
ac.finish_sentence(sentence, 3, corpus, deterministic=True)

['she', 'was', 'not', 'in', 'the', 'world', '.']

****